In [1]:
import pandas as pd

In [2]:
campione = pd.read_excel("./campione_400.xlsx")
print("Numero Coppie nel Campione:", len(campione))
print("Numero Coppie 'Matching' nel Campione:", len(campione[campione['Label'] == 1]))
print("Numero Coppie 'Non Matching' nel Campione:", len(campione[campione['Label'] == 0]))

Numero Coppie nel Campione: 400
Numero Coppie 'Matching' nel Campione: 173
Numero Coppie 'Non Matching' nel Campione: 227


# Valutazione del Blocking con Word2Vec
Calcoleremo Precision e Recall utilizzando il seguente formalismo:
- True Negative -> Coppie 'Non Matching' che non finiscono nello stesso Blocco
- False Positive -> Coppie 'Non Matching' che finiscono nello stesso Blocco
- False Negative -> Coppie 'Matching' che non finiscono nello stesso Blocco
- True Positive -> Coppie 'Matching' che finiscono nello stesso Blocco

In [12]:
schema_mediato_con_blocks = pd.read_csv("./../datasets/MediatedSchemaConBlocks_W2V.csv")

/Users/alessandropesare/anaconda3/envs/FinalProjectIDD3_8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: DtypeWarning: Columns (3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
schema_mediato_con_blocks

,name,location,marketCap,sharePrice,category,foundationYear,combined,embedding,block
0,Lacework,"United States, San Jose",$8.3B,NaN,Cybersecurity,2015,"Lacework United States, San Jose $8.3B nan Cyb...",[ 3.95248145e-01 6.27044514e-02 -1.41020780e-...,413
1,Tipalti,"United States, San Mateo",$8.3B,NaN,Fintech,2010,"Tipalti United States, San Mateo $8.3B nan Fin...",[ 3.95827174e-01 9.23668295e-02 3.06928111e-...,413
2,Tempus,"United States, Chicago",$8.1B,NaN,Health,2015,"Tempus United States, Chicago $8.1B nan Health...",[ 3.74361575e-01 1.75208226e-02 -6.78733736e-...,68
3,Anduril,"United States, Irvine",$8.48B,NaN,Artificial intelligence,2017,"Anduril United States, Irvine $8.48B nan Artif...",[ 3.04596931e-01 4.80266362e-02 -3.74700315e-...,68
4,Bolt,"Estonia, Tallinn",$8.4B,NaN,Auto & transportation,2013,"Bolt Estonia, Tallinn $8.4B nan Auto & transpo...",[ 1.50430694e-01 -4.03543971e-02 5.22575602e-...,849
...,...,...,...,...,...,...,...,...,...
81701,PepsiCo,"Purchase, NY, United States",$241.05 Billion,NaN,NaN,NaN,"PepsiCo Purchase, NY, United States $241.05 Bi...",[ 0.33947957 0.06857674 0.07836795 0.291687...,83
81702,Costco,"Issaquah, WA, United States",$258.15 Billion,NaN,NaN,NaN,"Costco Issaquah, WA, United States $258.15 Bil...",[ 3.73518854e-01 4.05892469e-02 1.02169275e-...,83
81703,Broadcom,"San Jose, CA, United States",$265.27 Billion,NaN,NaN,NaN,"Broadcom San Jose, CA, United States $265.27 B...",[ 0.41305602 0.11504476 0.08066246 0.273640...,739
81704,Coca-Cola,"Atlanta, GA, United States",$273.20 Billion,NaN,NaN,NaN,"Coca-Cola Atlanta, GA, United States $273.20 B...",[ 3.75501364e-01 7.65166506e-02 7.32515082e-...,83


In [14]:
tn, fp, fn, tp = list(), list(), list(), list()

for i in range(0, len(campione)):
    row = campione.iloc[i]
    first, second, true_label = row['Riga_SX'], row['Riga_DX'], row['Label']

    first_block = schema_mediato_con_blocks.iloc[first]['block']
    second_block = schema_mediato_con_blocks.iloc[second]['block']

    computed_label = None
    if first_block == second_block:
        computed_label = 1
    else:
        computed_label = 0
    
    if true_label == 0 and computed_label == 0:
        tn.append((first, second))
    if true_label == 0 and computed_label == 1:
        fp.append((first, second))
    if true_label == 1 and computed_label == 0:
        fn.append((first, second))
    if true_label == 1 and computed_label == 1:
        tp.append((first, second))

In [15]:
recall = len(tp) / (len(tp) + len(fn))
precision = len(tp) / (len(tp) + len(fp))
f_measure = (2 * precision * recall) / (precision + recall)

print("Recall:", recall)
print("Precision:", precision)
print("F-Measure:", f_measure)

Recall: 0.18497109826589594
Precision: 0.6666666666666666
F-Measure: 0.28959276018099545


# Valutazione del Pairwise Matching
Calcoleremo Precision e Recall utilizzando il seguente formalismo:
- True Negative -> Coppie 'Non Matching' che non finiscono nello stesso Clusterino
- False Positive -> Coppie 'Non Matching' che finiscono nello stesso Clusterino
- False Negative -> Coppie 'Matching' che non finiscono nello stesso Clusterino
- True Positive -> Coppie 'Matching' che finiscono nello stesso Clusterino

In [63]:
tn1, fp1, fn1, tp1 = list(), list(), list(), list()

In [61]:
results = dict()

# Apre il file in modalità di lettura
with open("./../3 - Pairwise Matching/results_clustersW2V.txt", 'r') as file:
    # Legge le righe del file
    lines = file.readlines()

i = 0
for line in lines:
    sep_index = line.find(':')
    key = int(line[:sep_index])
    
    if i != len(lines)-1:
        value = eval(line[sep_index+2:-1])
    else:
        value = eval(line[sep_index+2:])
    
    results[key] = value
    i += 1

In [64]:
## DA ULTIMARE!